In [5]:
import tellurium as te
import tesbml


def get_reaction_names(listOfReactions, reactionIDs):
    return [listOfReactions.get(id).getName() for id in reactionIDs]


def get_metabolite_names(listOfMetabolites, metaboliteIDs):
    return [listOfMetabolites.get(id).getName() for id in metaboliteIDs]


# We start with the smallest subsystem: Amino Sugar and Nucleotide Sugar Metabolism
asansm_sbml = "xml/iPAE1146_Amino_sugar_and_nucleotide_sugar_metabolism.xml"
asansm_libsbml_doc = tesbml.readSBML(asansm_sbml)
asansm_libsbml = asansm_libsbml_doc.getModel()
asansm_model = te.loadSBMLModel(asansm_sbml)
S_asansm = asansm_model.getFullStoichiometryMatrix()
S_asansm.colnames = get_reaction_names(asansm_libsbml.getListOfReactions(), S_asansm.colnames)
S_asansm.rownames = get_metabolite_names(asansm_libsbml.getListOfSpecies(), S_asansm.rownames)

# Next subsystem: Pyrimidine Metabolism
pm_sbml = "xml/iPAE1146_Pyrimidine_metabolism.xml"
pm_libsbml_doc = tesbml.readSBML(pm_sbml)
pm_libsbml = pm_libsbml_doc.getModel()
pm_model = te.loadSBMLModel(pm_sbml)
S_pm = pm_model.getFullStoichiometryMatrix()
S_pm.colnames = get_reaction_names(pm_libsbml.getListOfReactions(), S_pm.colnames)
S_pm.rownames = get_metabolite_names(pm_libsbml.getListOfSpecies(), S_pm.rownames)

# Third one: Lipopolysaccharide Biosynthesis
lb_sbml = "xml/iPAE1146_Lipopolysaccharide_biosynthesis.xml"
lb_libsbml_doc = tesbml.readSBML(lb_sbml)
lb_libsbml = lb_libsbml_doc.getModel()
lb_model = te.loadSBMLModel(lb_sbml)
S_lb = lb_model.getFullStoichiometryMatrix()
S_lb.colnames = get_reaction_names(lb_libsbml.getListOfReactions(), S_lb.colnames)
S_lb.rownames = get_metabolite_names(lb_libsbml.getListOfSpecies(), S_lb.rownames)

# Todo next: Get Fluxes v (to formulate dx=Sv)


In [39]:
import numpy as np
from sympy import Matrix

# return a dictionary 'index -> conservative relation'

def find_conservative_relations(S_matrix):
    
    S_matrix_transpose = np.transpose(S_matrix)
    S_matrix_left_null_space = Matrix(S_matrix_transpose).nullspace()
    
    number_of_conservative_relations = len(S_matrix_left_null_space)
    conservative_relationship_dict = {}
    
    for i in range(number_of_conservative_relations):
        counter = 0
        conservative_relationship_dict[i] = ''
        for metabolite in S_matrix.rownames:
            if S_matrix_left_null_space[i][counter] != 0:
                conservative_relationship_dict[i] += ' +('
                conservative_relationship_dict[i] += str(int(S_matrix_left_null_space[i][counter]))
                conservative_relationship_dict[i] += ')"'
                conservative_relationship_dict[i] += metabolite
                conservative_relationship_dict[i] += '"'
            counter += 1
        conservative_relationship_dict[i] += ' = 0'
    return conservative_relationship_dict

In [40]:
from pprint import PrettyPrinter

pp = PrettyPrinter()

print("\nThe conservative relations in the Amino Sugar and Nucleotide Sugar Metabolism subsystem are: \n")
pp.pprint(find_conservative_relations(S_asansm))

print("\nThe conservative relations in the Pyrimidine Metabolism subsystem are: \n")
pp.pprint(find_conservative_relations(S_pm))

print("\nThe conservative relations in the Lipopolysaccharide Biosynthesis subsystem are: \n")
pp.pprint(find_conservative_relations(S_lb))


The conservative relations in the Amino Sugar and Nucleotide Sugar Metabolism subsystem are: 

{0: ' +(1)"NAD" +(1)"NADH" = 0',
 1: ' +(1)"NADP" +(1)"NADPH" = 0',
 2: ' +(1)"CoA" +(1)"Acetyl-CoA" = 0',
 3: ' +(1)"NAD" +(-1)"H+" +(-1)"NADP" +(1)"CoA" +(1)"Phosphate" = 0',
 4: ' +(1)"NAD" +(1)"H2O" +(1)"Acetate" = 0',
 5: ' +(1)"NAD" +(1)"UDP-N-acetyl fucosamine" = 0',
 6: ' +(-1)"NADP" +(1)"UDP-MurNAc" = 0',
 7: ' +(1)"NAD" +(-1)"H+" +(1)"CoA" +(1)"UDP-N-acetylglucosamine enolpyruvate" '
    '= 0',
 8: ' +(1)"ACP" +(1)"Acetyl-ACP" = 0',
 9: ' +(-1)"NAD" +(1)"H+" +(1)"NADP" +(-1)"CoA" +(1)"Phosphoenolpyruvate" = 0',
 10: ' +(1)"ACP" +(1)"UDP-N-acetyl-D-mannosamine (2NAc3N) acid" = 0',
 11: ' +(-1)"ACP" +(1)"UDP-N-acetyl-D-manosamine (2NAc3NAc) acid" = 0',
 12: ' +(-1)"NAD" +(-1)"H2O" +(1)"N-Acetyl-D-glucosamine 6-phosphate" = 0',
 13: ' +(1)"PPi" +(1)"UTP" = 0',
 14: ' +(-2)"NAD" +(1)"H+" +(1)"NADP" +(-1)"CoA" '
     '+(1)"UDP-N-acetyl-D-galactosamine" +(-1)"PPi" '
     '+(1)"UDP-N-acet